# **Bert (Question Answering)**




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer


  

## Download Dataset

In [ ]:
!nvidia-smi

Wed May 19 02:52:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive

Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
!ls
%cd drive/MyDrive/ML/HW7_ChineseQA/
!ls

drive  sample_data
/content/drive/MyDrive/ML/HW7_ChineseQA
HW7_BertQA
hw7_data.zip
hw7_dev.json
hw7_test.json
hw7_train.json
result2.csv
result.csv
roberta_wwm_large_256_128_saved_model_1
roberta_wwm_large_256_128_saved_model_2
roberta_wwm_large_256_128_saved_model_3
roberta_wwm_large_256_128_saved_model_4
roberta_wwm_large_256_128_saved_model_5
roberta_wwm_large_384_192_saved_model_1
roberta_wwm_large_384_192_saved_model_2
roberta_wwm_large_384_192_saved_model_5
roberta_wwm_large_384_192_warm1000_saved_model_1
roberta_wwm_large_384_192_warm1000_saved_model_2
roberta_wwm_large_384_192_warm1000_saved_model_3
roberta_wwm_large_mix_dev_saved_model_1
roberta_wwm_large_mix_dev_saved_model_2
roberta_wwm_large_mix_dev_saved_model_3
roberta_wwm_large_mix_dev_saved_model_4
vocab.txt


In [ ]:
# Download link 1
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/drive/MyDrive/ML/HW7_ChineseQA/hw7_data.zip
7.71MB [00:00, 68.0MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          
Thu May  6 12:46:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [ ]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 4.3MB/s 
     |████████████████████████████████| 3.3MB 44.4MB/s 
     |████████████████████████████████| 901kB 49.5MB/s 


## Import Packages

In [ ]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset , ConcatDataset
from transformers import AdamW, XLNetForQuestionAnswering, XLNetForQuestionAnsweringSimple, XLNetTokenizerFast,XLNetTokenizer
from transformers import BertForQuestionAnswering, AutoModelForQuestionAnswering, BertTokenizerFast, AlbertForQuestionAnswering
from transformers import RobertaTokenizerFast, ElectraTokenizerFast
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm


device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [ ]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = False

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

In [ ]:
## check model dict to handle [UNK]
vocab = {}
with open('./vocab.txt' ,'r') as f:
    for word in f.readlines():
        word  = word.replace('\n','')
        vocab[word] = 0
print(len(vocab))

21128


## Load Model and Tokenizer




 

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-roberta-wwm-ext-large").to(device)
tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-roberta-wwm-ext-large")

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [ ]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [ ]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

In [ ]:
## make validation set [UNK] map
dev_unk_list = []
for idx,dpt in enumerate(dev_paragraphs):
    dev_unk = {}
    for word in dpt:
        if word not in vocab:
            dev_unk[word] = dev_paragraphs[idx].find(word)
    dev_unk_list.append(dev_unk)
print(dev_unk_list)

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'ㄑ': 18}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'C': 184}, {'—': 196}, {}, {}, {}, {}, {}, {}, {}, {}, {'–': 156, '℉': 177}, {'笪': 86}, {'圬': 256}, {}, {'舢': 76}, {}, {}, {}, {}, {}, {}, {'—': 141}, {}, {'K': 177}, {}, {'—': 276}, {}, {}, {}, {}, {'—': 7}, {}, {}, {}, {'甂': 157, '鯇': 260}, {'硤': 37, '芰': 217, '嚶': 347}, {}, {}, {'蘗': 57, '櫱': 255}, {}, {'秈': 211}, {'秈': 146}, {}, {}, {'簑': 287}, {}, {'秈': 9}, {'秈': 200}, {'粺': 80}, {}, {}, {}, {}, {'垓': 252}, {}, {}, {'–': 27}, {}, {'—': 254}, {}, {}, {}, {'堊': 42}, {}, {'堊': 1}, {}, {}, {'—': 34}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'—': 121}, {}, {}, {}, {}, {'烴': 47}, {}, {}, {}, {}, {}, {}, {'鉬': 195, '鎢': 203, '銻': 205}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'扃': 124}, {}, {}, {'暎': 226}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'谿': 133}, {'\u3000': 66, '毌': 257, '頎': 321}, {'貊': 126

In [ ]:
## make test set [UNK] map
test_unk_list = []
for idx,tpt in enumerate(test_paragraphs):
    test_unk = {}
    for word in tpt:
        if word not in vocab:
            test_unk[word] = test_paragraphs[idx].find(word)
    test_unk_list.append(test_unk)
print(test_unk_list)

[{'闥': 44}, {}, {'—': 264}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'迨': 83, '…': 292}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'帊': 42}, {}, {}, {}, {'—': 230}, {}, {}, {}, {}, {}, {}, {'鱒': 124}, {'僂': 21}, {'綷': 198}, {}, {}, {}, {}, {}, {}, {'—': 79}, {}, {'紇': 111}, {}, {}, {}, {}, {}, {'—': 408}, {}, {}, {'—': 31}, {}, {}, {}, {}, {}, {}, {}, {'銻': 115}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'稃': 43}, {}, {'㎝': 301}, {'癟': 132}, {}, {}, {}, {'秈': 77}, {'飧': 121, '糭': 273}, {}, {'秈': 2, '稉': 136}, {}, {'―': 161}, {}, {}, {}, {}, {}, {'–': 232}, {'—': 48}, {'—': 71, '咇': 243}, {'咇': 256}, {}, {}, {}, {'鈹': 143}, {}, {}, {'—': 255}, {}, {'堊': 6}, {}, {}, {}, {}, {}, {}, {}, {}, {'銥': 28}, {}, {}, {}, {'倻': 83, '拏': 130}, {}, {}, {'櫪': 11, '椴': 18, '櫟': 55, '拏': 81, '狍': 165, '猞': 175, '猁': 176}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'ㄙ': 160}, {}, {}, {}, {}, {'駢': 172}, {}, {}, {}, {

## Dataset and Dataloader

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 96
        self.max_paragraph_len = 384
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 192

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train" or self.split == "dev":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])
            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            ans_len = (answer_end_token - answer_start_token) 
            paragraph_start = max(0,answer_start_token - random.randint(0,self.max_paragraph_len - ans_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask


train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 1

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [ ]:
######### backup

def evaluate(data, output,mode,idx,q_len):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    if mode == 'dev':
        paragraph = dev_paragraphs[idx]
        unk_list = dev_unk_list
    elif mode == 'test':
        paragraph = test_paragraphs[idx]
        unk_list = test_unk_list
    answer = ''
    max_prob = -100.0
    num_of_windows = data[0].shape[1]
    ans_windows = 0
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        start_index_backup = start_index
        start_prob_backup = start_prob
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        end_index_backup = end_index
        end_prob_backup = end_prob
        end_tmp = output.end_logits[k]
        start_tmp = output.start_logits[k]
        count = 5

        ### check end
        if end_index < start_index: 
            end_tmp[end_index] = -10.0
            start_tmp[start_index] = -10.0
            sec_end_prob, sec_end_index = torch.max(end_tmp, dim=0)
            sec_start_prob, sec_start_index = torch.max(start_tmp, dim=0)
            if sec_start_prob + end_prob > start_prob + sec_end_prob and sec_start_index < end_index:
                start_index = sec_start_index
                start_prob = sec_start_prob
            elif sec_start_prob + end_prob < start_prob + sec_end_prob and start_index < sec_end_index:
                end_index = sec_end_index
                end_prob = sec_end_prob
            else:
                for c in range(count):
                    if end_index < start_index: ##max find 5 time
                        end_tmp[end_index] = -10.0
                        end_prob, end_index = torch.max(end_tmp, dim=0)
                    else:
                        break
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            ans_windows = k
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    answer = answer.replace(' ','')
    ## 處理UNK
    while '[UNK]' in answer:
        u_pos = answer.find('[UNK]')
        s_pos = ans_windows * 200 + start_index - q_len - 2
        u_pos += s_pos ### [UNK]'s actual position
        distance = 1000
        used_word = ''
        for i, word in enumerate(unk_list[idx]):
            if abs(u_pos - unk_list[idx][word]) < distance:
                distance = abs(u_pos - unk_list[idx][word])
                used_word = word
        answer = answer.replace('[UNK]',used_word,1)
    ## 處理上下引號
    if '《' in answer and '》' not in answer:
        answer += '》'
    elif '《' not in answer and '》' in answer:
        answer = '《' + answer
    if '「' in answer and '」' not in answer:
        answer += '」'
    elif '「' not in answer and '」' in answer:
        answer = '「' + answer
    return answer

In [ ]:
def evaluate_ensemble(data, outputs ,mode,idx,q_len):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    output = outputs[0]
    for out in range(len(outputs) - 1):
        output.start_logits += outputs[out + 1].start_logits
        output.end_logits += outputs[out + 1].end_logits
    if mode == 'dev':
        paragraph = dev_paragraphs[idx]
        unk_list = dev_unk_list
    elif mode == 'test':
        paragraph = test_paragraphs[idx]
        unk_list = test_unk_list
    answer = ''
    max_prob = -100.0
    num_of_windows = data[0].shape[1]
    ans_windows = 0
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        start_index_backup = start_index
        start_prob_backup = start_prob
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        end_index_backup = end_index
        end_prob_backup = end_prob
        end_tmp = output.end_logits[k]
        start_tmp = output.start_logits[k]
        count = 5

        ### check end
        if end_index < start_index: ##max find 20 time
            end_tmp[end_index] = -10.0
            start_tmp[start_index] = -10.0
            sec_end_prob, sec_end_index = torch.max(end_tmp, dim=0)
            sec_start_prob, sec_start_index = torch.max(start_tmp, dim=0)
            if sec_start_prob + end_prob > start_prob + sec_end_prob and sec_start_index < end_index:
                start_index = sec_start_index
                start_prob = sec_start_prob
            elif sec_start_prob + end_prob < start_prob + sec_end_prob and start_index < sec_end_index:
                end_index = sec_end_index
                end_prob = sec_end_prob
            else:
                for c in range(count):
                    if end_index < start_index: ##max find 5 time
                        end_tmp[end_index] = -10.0
                        end_prob, end_index = torch.max(end_tmp, dim=0)
                    else:
                        break

        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            ans_windows = k
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    answer = answer.replace(' ','')
    while '[UNK]' in answer:
        u_pos = answer.find('[UNK]')
        s_pos = ans_windows * 200 + start_index - q_len - 2
        u_pos += s_pos ### [UNK]'s actual position
        distance = 1000
        used_word = ''
        for i, word in enumerate(unk_list[idx]):
            if abs(u_pos - unk_list[idx][word]) < distance:
                distance = abs(u_pos - unk_list[idx][word])
                used_word = word
        answer = answer.replace('[UNK]',used_word,1)

     ## 處理上下引號
    if '《' in answer and '》' not in answer:
        answer += '》'
    elif '《' not in answer and '》' in answer:
        answer = '《' + answer
    if '「' in answer and '」' not in answer:
        answer += '」'
    elif '「' not in answer and '」' in answer:
        answer = '「' + answer

    return answer

## Training

In [ ]:
num_model = 3
num_epoch = 3
validation = True
logging_step = 100
learning_rate = 3e-5
accum_iter = 48
optimizer = AdamW(model.parameters(), lr=learning_rate,betas = (0.9,0.99),eps = 1e-6 ) ##test看看
total_step = len(train_loader) * num_epoch / accum_iter
warm_step = total_step * 0.06
schedular = get_linear_schedule_with_warmup(optimizer,warm_step,total_step)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 
step = 1
model.train()

print("Start Training ...")
for num in range(num_model):
    best_acc = 0
    if num !=0:
        model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-roberta-wwm-ext-large").to(device)
        optimizer = AdamW(model.parameters(), lr=learning_rate,betas = (0.9,0.98),eps = 1e-6 ) 
        schedular = get_linear_schedule_with_warmup(optimizer,warm_step,total_step)
    step = 1
    for epoch in range(num_epoch):
        train_loss = train_acc = 0
        for batch_idx, data in enumerate(tqdm(train_loader)):	
            with torch.set_grad_enabled(True):
                # Load all data into GPU
                data = [i.to(device) for i in data]
                
                # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
                # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
                output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

                # Choose the most probable start position / end position
                start_index = torch.argmax(output.start_logits, dim=1)
                end_index = torch.argmax(output.end_logits, dim=1)
                
                # Prediction is correct only if both start_index and end_index are correct
                train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
                train_loss += output.loss
                
                if fp16_training:
                    accelerator.backward(output.loss)
                else:
                    output.loss.backward()

                if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
                    optimizer.step()
                    optimizer.zero_grad()
                    schedular.step()
                    step += 1

                ##### TODO: Apply linear learning rate decay #####
                # Print training loss and accuracy over past logging step
                if step % logging_step == 0 and ((batch_idx + 1) % accum_iter == 0):
                    print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / (logging_step * accum_iter):.3f}, acc = {train_acc / (logging_step * accum_iter):.3f}")
                    train_loss = train_acc = 0
        if validation:
            print("Evaluating Dev Set ...")
            model.eval()
            with torch.no_grad():
                dev_acc = 0
                for i, data in enumerate(tqdm(dev_loader)):
                    output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                          attention_mask=data[2].squeeze(dim=0).to(device))
                    # prediction is correct only if answer text exactly matches
                    dev_acc += evaluate(data, output ,'dev', dev_questions[i]["paragraph_id"],len(dev_questions[i]["question_text"])) == dev_questions[i]["answer_text"]
                print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
                if dev_acc > best_acc:
                    best_acc = dev_acc
                    print("Saving Model ...")
                    model_save_dir = "roberta_wwm_large_saved_model_" + str(num+1) 
                    model.save_pretrained(model_save_dir)
            model.train()
        # Save a model and its configuration file to the directory「saved_model」 
        # i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
        # Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
        

Start Training ...


Epoch 1 | Step 100 | loss = 2.774, acc = 0.354
Epoch 1 | Step 200 | loss = 0.600, acc = 0.747
Epoch 1 | Step 300 | loss = 0.533, acc = 0.780
Epoch 1 | Step 400 | loss = 0.487, acc = 0.792
Epoch 1 | Step 500 | loss = 0.482, acc = 0.796



Epoch 1 | Step 600 | loss = 0.452, acc = 0.808

Saving Model ...


Epoch 2 | Step 700 | loss = 0.164, acc = 0.561
Epoch 2 | Step 800 | loss = 0.275, acc = 0.865
Epoch 2 | Step 900 | loss = 0.246, acc = 0.874
Epoch 2 | Step 1000 | loss = 0.268, acc = 0.874
Epoch 2 | Step 1100 | loss = 0.266, acc = 0.872



Epoch 2 | Step 1200 | loss = 0.226, acc = 0.884

Saving Model ...


...

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext-large were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl

Epoch 1 | Step 100 | loss = 2.558, acc = 0.414
Epoch 1 | Step 200 | loss = 0.523, acc = 0.777
Epoch 1 | Step 300 | loss = 0.495, acc = 0.793
Epoch 1 | Step 400 | loss = 0.472, acc = 0.798
Epoch 1 | Step 500 | loss = 0.442, acc = 0.819



Epoch 1 | Step 600 | loss = 0.411, acc = 0.815

Saving Model ...


Epoch 2 | Step 700 | loss = 0.134, acc = 0.579
Epoch 2 | Step 800 | loss = 0.162, acc = 0.921
Epoch 2 | Step 900 | loss = 0.176, acc = 0.920
Epoch 2 | Step 1000 | loss = 0.184, acc = 0.914
Epoch 2 | Step 1100 | loss = 0.182, acc = 0.917



Epoch 2 | Step 1200 | loss = 0.174, acc = 0.905

Saving Model ...


In [ ]:
model2 = AutoModelForQuestionAnswering.from_pretrained("roberta_wwm_large_saved_model_1").to(device)
model3 = AutoModelForQuestionAnswering.from_pretrained("roberta_wwm_large_saved_model_2").to(device)

In [ ]:
## ensemble
print("Evaluating Dev Set ...")
model.eval()
model2.eval()
model3.eval()
epoch = 0
with torch.no_grad():
    dev_acc = 0
    for i, data in enumerate(tqdm(dev_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output2 =  model2(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output3 =  model3(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
        output_list = [output,output2,output3]
        dev_acc += evaluate_ensemble(data, output_list, 'dev', dev_questions[i]["paragraph_id"],len(dev_questions[i]["question_text"])) == dev_questions[i]["answer_text"]
    print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")

Evaluating Dev Set ...



Validation | Epoch 1 | acc = 0.848


## Testing

In [ ]:
print("Evaluating Test Set ...")

result = []

model.eval()
model2.eval()
model3.eval()

epoch = 0
with torch.no_grad():
    dev_acc = 0
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output2 =  model2(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output3 =  model3(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
        output_list = [output,output2,output3]
        result.append(evaluate_ensemble(data, output_list, 'test', test_questions[i]["paragraph_id"],len(test_questions[i]["question_text"])))

result_file = "result_ensemble.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...



Completed! Result is in result_mix.csv
